# Run Functions to Add Information to Projects

In [1]:
# ! pip install nltk

In [2]:
import numpy as np
import pandas as pd
from siuba import *

from calitp_data_analysis.sql import to_snakecase

from shared_utils import geography_utils

import dla_utils

import utils

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [3]:
pd.set_option("display.max_columns", 100)
pd.set_option('display.max_colwidth', None)

## Read in Data and function development

In [4]:
GCS_FILE_PATH  = 'gs://calitp-analytics-data/data-analyses/dla/dla-iija'

In [5]:
### Read in data (First round)
# proj = to_snakecase(pd.read_excel(f"{GCS_FILE_PATH}/FMIS_Projects_Universe_(IIJA_Reporting)_ (3-9-23).xlsx", header=[3], skiprows=range(4,1784)))
# proj.drop(columns =['unnamed:_0'], axis=1, inplace=True)
# proj['summary_recipient_defined_text_field_1_value'] = proj['summary_recipient_defined_text_field_1_value'].fillna(value='None')


In [6]:
proj = to_snakecase(pd.read_excel(f"{GCS_FILE_PATH}/IIJA_FMIS_AllProject_20230309_ToDLA.xlsx",))


In [7]:
proj.sample()

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,project_status_description,project_description,improvement_type,improvement_type_description,total_cost_amount,obligations_amount,summary_recipient_defined_text_field_1_value
526,44694,Y400,CONGESTION MITIGATION IIJA,5942284,0617000209L,AMERICAN AVENUE FROM MADERA AVENUE TO PLACER AVENUE SHOULDER IMPROVEMENTS,19,Cong Dist 21,Active,AMERICAN AVENUE FROM MADERA AVENUE TO PLACER AVENUE SHOULDER IMPROVEMENTS,17,Construction Engineering,285000.0,252310.0,L5942COFCG


In [9]:
# proj.congressional_district.

In [10]:
new_codes = to_snakecase(pd.read_excel(f"{GCS_FILE_PATH}/FY21-22ProgramCodesAsOf5-25-2022.v2.xlsx"))
code_map = dict(new_codes[['iija_program_code', 'new_description']].values)


In [11]:
new_codes

,description,iija_program_code,new_description
0,Advanced Transportation Technologies and Innovative Mobility Deployment,Y44A,Advanced Transportation Technologies and Innovative Mobility Deployment
1,Applalachian Developmen Highway System,Y140,Applalachian Developmen Highway System
2,Border State Infrastructure,Y500,Border State Infrastructure
3,Bridge Formula Program,Y110,Bridge Formula Program
4,Bridge Formula Program Off-System Bridges,Y120,Bridge Formula Program
...,...,...,...
96,Transportation Alternatives (Section 133(h)) - Areas > 200k STBG Program Eligibilities,Y304,Transportation Alternatives (Section 133(h))
97,Transportation Alternatives (Section 133(h)) - Areas with Population 50K to 200K [NEW],Y306,Transportation Alternatives (Section 133(h))
98,"Transportation Alternatives (Section 133(h)) - Areas with Population 5K to 49,999 [NEW]",Y307,Transportation Alternatives (Section 133(h))
99,Transportation Alternatives (Section 133(h)) - Areas with Population Less than 5K [NEW],Y308,Transportation Alternatives (Section 133(h))


In [12]:
proj['program_code_description'] = proj.program_code.map(code_map)

In [13]:
proj.sample()

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,project_status_description,project_description,improvement_type,improvement_type_description,total_cost_amount,obligations_amount,summary_recipient_defined_text_field_1_value
1222,44816,Y306,Transportation Alternatives (Section 133(h)),5382038,1120000031L,THE TOWNSITE DRIVE COMPLETE STREETS PROJECT IS LOCATED IN THE CITY OF VISTA ALONG THE SEGMENT OF NORTH DRIVE FROM JUST NORTHWEST OF THE INTERSECTION,73,Cong Dist 49,Active,"THE TOWNSITE DRIVE COMPLETE STREETS PROJECT IS LOCATED IN THE CITY OF VISTA ALONG THE SEGMENT OF NORTH DRIVE FROM JUST NORTHWEST OF THE INTERSECTION OF EAST DRIVE TO THE INTERSECTION OF WEST LOS ANGELES DRIVE; ALONG THE SEGMENT OF WEST LOS ANGELES DRIVE FROM THE INTERSECTION OF NORTH DRIVE TO THE INTERSECTION OF NORTH SANTA FE AVENUE; AND ALONG THE SEGMENT OF TOWNSITE DRIVE FROM THE INTERSECTION OF NORTH SANTA FE AVENUE TO THE INTERSECTION OF EAST VISTA WAY. THE PROJECT WILL TRANSFORM TOWNSITE DRIVE AND W. LOS ANGELES DRIVE TO COMPLETE STREETS ACCOMMODATING ALL USERS BY CONSTRUCTING SIDEWALKS, TRAFFIC CIRCLES, CURB EXTENSIONS, BIKE LANES, SHARROWS AND HIGH VISIBILITY CROSSWALKS. THE PROJECT WILL ALSO CONSTRUCT CURB EXTENSIONS, HIGH VISIBILITY CROSSWALK, GREEN BIKE LANE EXTENSIONS THROUGH INTERSECTIONS AND TWO MID BLOCK PEDESTRIAN TRAFFIC SIGNALS ON N. SANTA FE AVENUE BETWEEN CALIFORNIA AVE AND BOBIER DR.",28,Facilities for Pedestrians and Bicycles,3651000.0,3232230.0,L5382SANDAG


In [14]:
proj['summary_recipient_defined_text_field_1_value'] = proj['summary_recipient_defined_text_field_1_value'].astype(str)

In [15]:

# df = utils.read_data_all()
    
    ## function that adds known agency name to df 
df = utils.identify_agency(proj, 'summary_recipient_defined_text_field_1_value')

In [16]:
df.sample(3)

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,project_status_description,project_description,improvement_type,improvement_type_description,total_cost_amount,obligations_amount,summary_recipient_defined_text_field_1_value,implementing_agency_locode,implementing_agency,district,county_name,rtpa_name,mpo_name
425,44686,YS32,Section 164 Penalties - Use for HSIP Activities,P135009,0517000184S,"SANTA BARBARA COUNTY IN SANTA MARIA AT BUNNY AVENUE AND FROM ROEMER WAY TO PREISKER LANE. INSTALL SIGNAL, LIGHTING, CURB RAMPS AND SIDEWALK.",83,Cong Dist 24,Active,"ON STATE ROUTE: 135. SANTA BARBARA COUNTY IN SANTA MARIA AT BUNNY AVENUE AND FROM ROEMER WAY TO PREISKER LANE. INSTALL SIGNAL, LIGHTING, CURB RAMPS AND SIDEWALK.",21,Safety,1269500.0,1087000.0,S SBCAG,NaN,California,5.0,Santa Barbara County,NaN,NaN
551,44699,Y001,National Highway Performance Program (NHPP),P099661,1014000134S,"SAN JOAQUIN COUNTY IN LODI FROM 0.4 MILE SOUTH OF ROUTE 12/99 SEPARATION TO 0.3 MILE SOUTH OF TURNER ROAD OVERCROSSING. RAISE BRIDGES, REALIGN RAMPS",77,Cong Dist 9,Active,"ON STATE ROUTE: 99. SAN JOAQUIN COUNTY IN LODI FROM 0.4 MILE SOUTH OF ROUTE 12/99 SEPARATION TO 0.3 MILE SOUTH OF TURNER ROAD OVERCROSSING. RAISE BRIDGES, REALIGN RAMPS AND CONSTRUCT AUXILIARY LANE.",17,Construction Engineering,460000.0,407238.0,S SJCOG,NaN,California,10.0,San Joaquin County,NaN,NaN
239,44684,ER01,NaN,32L0081,0517000180L,"BROWNS VALLEY ROAD PM 2.14 REPAIR EMBANKMENT, CRIB WALL AND ROADWAY",87,Cong Dist 20,Active,"BROWNS VALLEY ROAD PM 2.14 REPAIR EMBANKMENT, CRIB WALL AND ROADWAY",4,4R - No Added Capacity,82860.0,73355.0,L5936AMBAG,5936.0,Santa Cruz County,5.0,Santa Cruz County,Santa Cruz County Regional Transportation Commission,Association of Monterey Bay Area Governments


In [17]:
len(df>>filter(_.implementing_agency=='Unknown'))

1

In [18]:
len(df>>filter(_.county_name=='Unknown'))

1

In [19]:
df>>filter(_.implementing_agency=="Unknown")

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,project_status_description,project_description,improvement_type,improvement_type_description,total_cost_amount,obligations_amount,summary_recipient_defined_text_field_1_value,implementing_agency_locode,implementing_agency,district,county_name,rtpa_name,mpo_name
706,44827,YR10,State RTP Administration,2023000,NaN,2023 Rec Trails Project: State Parks Incurred Administrative Expenses,999,Cong Dist 0,Active,"2023 Rec Trails Project: State Parks Incurred Administrative Expenses. State costs incurred administering the RTP such as staff time, meeting and travel costs and attendance at trail-related training sessions and conferences, Trail Advisory committee costs including meeting and travel costs for committee members, trail conference support, etc.",44,Other,274727.3,241760.0,nan,NaN,Unknown,NaN,Unknown,NaN,NaN


In [20]:
df>>filter(_.project_title.str.contains("Administrative Expense"))

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,project_status_description,project_description,improvement_type,improvement_type_description,total_cost_amount,obligations_amount,summary_recipient_defined_text_field_1_value,implementing_agency_locode,implementing_agency,district,county_name,rtpa_name,mpo_name
706,44827,YR10,State RTP Administration,2023000,NaN,2023 Rec Trails Project: State Parks Incurred Administrative Expenses,999,Cong Dist 0,Active,"2023 Rec Trails Project: State Parks Incurred Administrative Expenses. State costs incurred administering the RTP such as staff time, meeting and travel costs and attendance at trail-related training sessions and conferences, Trail Advisory committee costs including meeting and travel costs for committee members, trail conference support, etc.",44,Other,274727.3,241760.0,nan,NaN,Unknown,NaN,Unknown,NaN,NaN


## Test & Export

In [21]:
# GCS_FILE_PATH  = 'gs://calitp-analytics-data/data-analyses/dla/dla-iija'

In [22]:
df.sample()

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,project_status_description,project_description,improvement_type,improvement_type_description,total_cost_amount,obligations_amount,summary_recipient_defined_text_field_1_value,implementing_agency_locode,implementing_agency,district,county_name,rtpa_name,mpo_name
463,44690,Y001,National Highway Performance Program (NHPP),0052030,0716000028S,IN LOS ANGELES COUNTY IN LOS ANGELES FROM ROUTE 5/118 SEPARATION TO BALBOA BOULEVARD OVERCROSSINGROADSIDE SAFETY IMPROVEMENTS : CONSTRUCT MAINTENANCE,37,Cong Dist 30,Active,"ON STATE ROUTE: 5. IN LOS ANGELES COUNTY IN LOS ANGELES FROM ROUTE 5/118 SEPARATION TO BALBOA BOULEVARD OVERCROSSINGROADSIDE SAFETY IMPROVEMENTS : CONSTRUCT MAINTENANCE VEHICLE PULLOUTS (MVPS), ACCESS ROADS AND GATES, RELOCATE IRRIGATION FACILITIES, UPGRADE ELECTRICAL PULL BOXES, PAVE SLOPENARROW AREAS, BEYOND GORE ANDUNDER GUARDRAIL. IMPROVED ROADSIDE SAFETY",44,Other,16874.69,15019.8,S SCAG,NaN,California,7.0,Los Angeles County,NaN,NaN


In [23]:
###test agg. PASS df with title column, concat and 552 rows
##throwingerror
agg = utils.get_clean_data(df, full_or_agg = 'agg')

In [35]:
agg.sample()

,fmis_transaction_date,project_number,implementing_agency,summary_recipient_defined_text_field_1_value,program_code,program_code_description,recipient_project_number,improvement_type,improvement_type_description,project_title,obligations_amount,congressional_district,district,county_code,county_name,county_name_title,implementing_agency_locode,rtpa_name,mpo_name,project_title_new
338,44733,P210017,California,S SCAG,Y001,National Highway Performance Program (NHPP),0716000178S,17|37|42,Construction Engineering|Mitigation of Water Pollution due to Highway Runoff|Training,"IN AND NEAR LA VERNE, AT VARIOUS LOCATIONS FROM 0.4 MILE EAST OF SOUTH LONE HILL AVENUE UNDERCROSSING TO COUNTY LINE. CONSTRUCT STORMWATER TREATMENT",4525800.0,Cong Dist 27|Cong Dist 32,7.0,37,Los Angeles County,Los Angeles County,nan,None,None,Construct Storm Water Mitigation in Los Angeles County


In [37]:
df.head()

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,project_status_description,project_description,improvement_type,improvement_type_description,total_cost_amount,obligations_amount,summary_recipient_defined_text_field_1_value,implementing_agency_locode,implementing_agency,district,county_name,rtpa_name,mpo_name,county_name_title
0,44594,ER01,nan,32L0118,0117000181L,MATTOLE ROAD PM 5.00 PERMANENT RESTORATION,23,Cong Dist 2,Active,MATTOLE ROAD PM 5.00 PERMANENT RESTORATION,4,4R - No Added Capacity,68757.00,60870.23,L5904NON-MPO,5904.0,Humboldt County,1.0,Humboldt County,Humboldt County Association of Governments,NON-MPO,Humboldt County
1,44594,ER01,nan,32L0118,0117000181L,MATTOLE ROAD PM 5.00 PERMANENT RESTORATION,23,Cong Dist 2,Active,MATTOLE ROAD PM 5.00 PERMANENT RESTORATION,15,Preliminary Engineering,207848.00,184008.19,L5904NON-MPO,5904.0,Humboldt County,1.0,Humboldt County,Humboldt County Association of Governments,NON-MPO,Humboldt County
2,44594,ER01,nan,32L0118,0117000181L,MATTOLE ROAD PM 5.00 PERMANENT RESTORATION,23,Cong Dist 2,Active,MATTOLE ROAD PM 5.00 PERMANENT RESTORATION,17,Construction Engineering,139775.00,123742.50,L5904NON-MPO,5904.0,Humboldt County,1.0,Humboldt County,Humboldt County Association of Governments,NON-MPO,Humboldt County
3,44594,ER01,nan,4400046,01924731L,ALDERPOINT ROAD PM 40.21 TO 40.70 EMERGENCY RELIEF PR ONLY,23,Cong Dist 2,Active,ALDERPOINT ROAD PM 40.21 TO 40.70 EMERGENCY RELIEF PR ONLY,4,4R - No Added Capacity,124980.90,110645.59,L5904NON-MPO,5904.0,Humboldt County,1.0,Humboldt County,Humboldt County Association of Governments,NON-MPO,Humboldt County
4,44594,ER01,nan,4400046,01924731L,ALDERPOINT ROAD PM 40.21 TO 40.70 EMERGENCY RELIEF PR ONLY,23,Cong Dist 2,Active,ALDERPOINT ROAD PM 40.21 TO 40.70 EMERGENCY RELIEF PR ONLY,15,Preliminary Engineering,831.49,736.11,L5904NON-MPO,5904.0,Humboldt County,1.0,Humboldt County,Humboldt County Association of Governments,NON-MPO,Humboldt County


In [22]:
sorted(list(agg.project_title_new.unique()))

[' Added Roadway Capacity in Marin County',
 ' Added Roadway Capacity in Orange County',
 ' Bike Lanes in Alameda',
 ' Bike Share Program in Windsor',
 ' Bike Share Program in the Metropolitan Transportation Commission Region',
 ' Bridge Preventive Maintenance in Humboldt County',
 ' Bridge Preventive Maintenance in Pleasanton',
 ' Bridge Rehabilitation in Chico',
 ' Bridge Rehabilitation in Contra Costa County',
 ' Bridge Rehabilitation in Los Angeles',
 ' Bridge Rehabilitation in Los Angeles County',
 ' Bridge Rehabilitation in Monterey County',
 ' Bridge Rehabilitation in Palm Springs',
 ' Bridge Rehabilitation in Sacramento County',
 ' Bridge Rehabilitation in San Benito County',
 ' Bridge Rehabilitation in San Diego',
 ' Bridge Rehabilitation in San Francisco County',
 ' Bridge Rehabilitation in San Joaquin County',
 ' Bridge Rehabilitation in San Luis Obispo County',
 ' Bridge Rehabilitation in Santa Cruz County',
 ' Bridge Rehabilitation in Sierra County',
 ' Bridge Rehabilitati

In [40]:
agg>>filter(_.implementing_agency.str.contains("California"))

,fmis_transaction_date,project_number,implementing_agency,summary_recipient_defined_text_field_1_value,program_code,program_code_description,recipient_project_number,improvement_type,improvement_type_description,project_title,obligations_amount,congressional_district,district,county_code,county_name,county_name_title,implementing_agency_locode,rtpa_name,mpo_name,project_title_new
0,44579,15D0001,California,S ER NONE,ER01,nan,0721000066S,6|17,4R - Restoration & Rehabilitation|Construction Engineering,"ON LA-39 PM 18.0/30.9 & 39.5/44.4 IN ANGELES NATIONAL FOREST, IN THE UNINCORPORATED AREA OF COUNTY OF LOS ANGELES EMERGENCY PROJECT - EMERGENCY OPENI",608700.00,Cong Dist 27|Cong Dist 32,7.0,37,Los Angeles County,Los Angeles County,nan,None,None,Road Restoration & Rehabilitation in Los Angeles County
1,44579,39BD001,California,S ER NONE,ER01,nan,0719000177S,6|17,4R - Restoration & Rehabilitation|Construction Engineering,"IN LOS ANGELES COUNTY, IN EACH DIRECTION ON LA-39 IN THE UNINCORPORATED AREA EMERGENCY PROJECT - EMERGENCY OPENING",176900.00,Cong Dist 27,7.0,37,Los Angeles County,Los Angeles County,nan,None,None,Road Restoration & Rehabilitation in Los Angeles County
2,44581,31RA002,California,S AMBAG,ER01,nan,0518000118S,16|43,Right of Way|Utilities,MONTEREY COUNTY NEAR BIG SUR 2.3 MILES NORTH OF CASTRO CANYON BRIDGE TO 0.8 MILE SOUTH OF BIG SUR RIVER BRIDGE. EMERGENCY PROJECT - PERMANENT RESTORA,2983400.00,Cong Dist 20,5.0,53,Monterey County,Monterey County,nan,None,None,Right of Way Project in Monterey County
3,44582,4332005,California,S ER NONE,ER01,nan,10061P043S,15,Preliminary Engineering,MPA CO; NON-INTERSTATE RTES AT VAR LOCATIONS PRELIMINARTY ENGINEERING,2041079.35,Cong Dist 4,10.0,43,Mariposa County,Mariposa County,nan,None,None,Preliminary Engineering Projects in Mariposa County
4,44582,ERI8005,California,S NON-MPO,ER01,nan,1020000099S,6|17|42,4R - Restoration & Rehabilitation|Construction Engineering|Training,IN MARIPOSA COUNTY ABOUT 8 MILES WEST OF EL PORTAL FROM 1.2 MILES WEST OF SOUTH FORK MERCED RIVER BRIDGE TO 0.5 MILE WEST OF SOUTH FORK MERCED RIVER,26862300.00,Cong Dist 16,10.0,47,Merced County,Merced County,nan,None,None,Road Restoration & Rehabilitation in Merced County
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1045,44991,P133016,California,S SCAG,YS30,Highway Safety Improvement Program (HSIP),1214000077S,21,Safety,"IN ORANGE COUNTY, IN AND NEAR LAGUNA BEACH AND IRVINE, FROM NORHT OF EL TORO ROAD TO RTE. 241. FLATTEN STEEP EMBANKMENTS, REPLACE AND UPGRADE GUARDRA",810000.00,Cong Dist 45|Cong Dist 48,12.0,59,Orange County,Orange County,nan,None,None,Safety Improvements in Orange County
1046,44991,X099646,California,S NON-MPO,Y240,Surface Transportation Block Grant (STBG) Program,1015000115S,17,Construction Engineering,IN AND NEAR THE CITY OF MODESTO ON SR 132 NEAR DAKOTA AVE TO NEAR 6TH ST AND ON SR 99 FROM 0.5 M SOUTH OF TUOLUMNE RIVER BRIDGE TO 0.1M NORTH OF WOOD,250000.00,Cong Dist 10,10.0,99,Stanislaus County,Stanislaus County,nan,None,None,Construction Engineering Projects in Stanislaus County
1047,44992,0802374,California,S MTC,Y001,National Highway Performance Program (NHPP),0417000031S,11|17|42,Bridge Replacement - No Added Capacity|Construction Engineering|Training,SOLANO COUNTY IN VALLEJO ON ROUTE 80 FROM 0.2 MILE NORH OF CARQUINEZ BRIDGE TO PLAZA TO 0.3 MILES SOUTH OF MAGAZINE STREET OVERCROSSING AND AT ROUTE,22715000.00,Cong Dist 5,4.0,95,Solano County,Solano County,nan,None,None,Bridge Replacement in Solano County
1049,44993,31TV004,California,S NON-MPO,ER01,nan,0218000119S,6|17|42,4R - Restoration & Rehabilitation|Construction Engineering|Training,IN PLUMAS COUNTY AT VARIOUS LOCATIONS FROM BUTTE COUNTY LINE TO 3.1 MILES WEST OF ROUTE 89 REPAIR ROCK SLOPE PROTECTION AND STABILIZE ROADWAY EMBANKM,20798717.60,Cong Dist 1,2.0,63,Plumas County,Plumas County,nan,None,None,Road Restoration & Rehabilitation in Plumas County


In [38]:
agg>>filter(_.project_title_new.str.contains("Caltrans"))

,fmis_transaction_date,project_number,implementing_agency,summary_recipient_defined_text_field_1_value,program_code,program_code_description,recipient_project_number,improvement_type,improvement_type_description,project_title,obligations_amount,congressional_district,district,county_code,county_name,county_name_title,implementing_agency_locode,rtpa_name,mpo_name,project_title_new
203,44686,6204134,Caltrans,S6204MTC,Y230,Surface Transportation Block Grant (STBG) Program,0419000044L,15,Preliminary Engineering,"ALONG THE I-880 CORRIDOR SF BAY AREA: REGIONWIDE: IMPLEMENT A COLLECTIVE APPROACH TO FREEWAY OPERATIONS AND MANAGEMENT, INCLUDING COMMUNICATIONS NETW",1740000.0,Cong Dist 13,4.0,1,Multi-County,Multi-County,6204.0,CT-ADMIN,CT-ADMIN,Caltrans Preliminary Engineering Projects
560,44815,6203056,Caltrans,S6203SACOG,Y400,Congestion Mitigation & Air Quality Improvement (CMAQ),0316000113L,15,Preliminary Engineering,"SR-51 (CAPITAL CITY) CORRIDOR IMPROVEMENTS FROM J ST TO ARDEN WAY. WIDEN AND CONSTRUCT BUS/CARPOOL LANES, TRANSITION/AUXILIARY & NB CLASS I BIKE PATH",2000000.0,Cong Dist 6,3.0,67,Multi-County,Multi-County,6203.0,CT-ADMIN,CT-ADMIN,Caltrans Construct Bike Lanes
589,44817,6207070,Caltrans,S6207SCAG,Y400,Congestion Mitigation & Air Quality Improvement (CMAQ),0713000007L,1|17,New Construction Roadway|Construction Engineering,RT. 10 : FROM CITRUS AVE. TO RT. 57 CONSTRUCT HOV LANE IN BOTH DIRECTIONS. (TC),8165020.0,Cong Dist 32|Cong Dist 35|Cong Dist 39,7.0,37,Multi-County,Multi-County,6207.0,CT-ADMIN,CT-ADMIN,Caltrans Construct HOV Lane
742,44879,6212023,Caltrans,S6212SCAG,Y230,Surface Transportation Block Grant (STBG) Program,1213000084L,15,Preliminary Engineering,INTERSTATE 5 FROM LOS ALISOS BOULEVARD OVERCROSSING TO THE RIDGE ROUTE DRIVE ALIGNMENT RAMP IMPROVEMENTS AT THE EL TORO ROAD INTERCHANGE (TC),4813000.0,Cong Dist 45,12.0,59,Multi-County,Multi-County,6212.0,CT-ADMIN,CT-ADMIN,Caltrans Improve Ramp
983,44971,6212028,Caltrans,S6212SCAG,Y230,Surface Transportation Block Grant (STBG) Program,1220000035L,16,Right of Way,I-5 FROM YALE AVENUE TO SR-55 ADD ONE MIXED-FLOW LANE IN BOTH THE NORTHBOUND AND SOUTHBOUND DIRECTIONS,1230000.0,Cong Dist 45,12.0,59,Multi-County,Multi-County,6212.0,CT-ADMIN,CT-ADMIN,Caltrans Right of Way Project
1029,44984,6212022,Caltrans,S6212SCAG,Y230,Surface Transportation Block Grant (STBG) Program,1214000097L,17,Construction Engineering,"INTERSTATE 5 FROM OSO CREEK TO ALICIA PARKWAY CONSTRUCT ONE GENERAL PURPOSE LANE ON EACH DIRECTION, RECONSTRUCT LA PAZ ROAD INTERCHANGE AND ADD AUXIL",1000000.0,Cong Dist 45,12.0,59,Multi-County,Multi-County,6212.0,CT-ADMIN,CT-ADMIN,Caltrans Construction Engineering Projects


In [24]:
len(agg)

1054

In [25]:
# agg = utils.title_column_names(agg)

In [26]:
# agg.to_csv(f"{GCS_FILE_PATH}/IIJA_FMIS_AllProject_20230315_ToDLA_agg.csv")

In [27]:
###test full. PASS title, no concat and 1241 rows
full = utils.get_clean_data(df, full_or_agg = 'full')

In [28]:
# full = utils.title_column_names(full)

In [29]:
# full.to_csv(f"{GCS_FILE_PATH}/FMIS_Projects_Universe_IIJA_Reporting_3-9-23_all.csv")

## Check Titles

In [30]:
# agg = to_snakecase(agg)

In [31]:
test_df = agg>>filter(_.implementing_agency.str.contains('Unknown')
                      | _.implementing_agency.str.contains("Caltrans")
                     )

In [32]:
test_df.sample()

,fmis_transaction_date,project_number,implementing_agency,summary_recipient_defined_text_field_1_value,program_code,program_code_description,recipient_project_number,improvement_type,improvement_type_description,project_title,obligations_amount,congressional_district,district,county_code,county_name,county_name_title,implementing_agency_locode,rtpa_name,mpo_name,project_title_new
589,44817,6207070,Caltrans,S6207SCAG,Y400,Congestion Mitigation & Air Quality Improvement (CMAQ),0713000007L,1 | 17,New Construction Roadway | Construction Engineering,RT. 10 : FROM CITRUS AVE. TO RT. 57 CONSTRUCT HOV LANE IN BOTH DIRECTIONS. (TC),8165020.0,Cong Dist 32 | Cong Dist 35 | Cong Dist 39,7.0,37,Multi-County,Multi-County,6207.0,CT-ADMIN,CT-ADMIN,Caltrans Construct HOV Lane


In [33]:
locodes = to_snakecase(pd.read_excel(f"gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/locodes_updated7122021.xlsx"))

In [34]:
locodes['agency_name'] = locodes['agency_name'].str.upper()

In [35]:
locode_names = sorted(list(locodes['agency_name'].unique()), reverse=True)

In [36]:
locode_names.remove('ROSS')


In [37]:
locode_names.append("State Parks")

In [38]:
locode_names

['YUCCA VALLEY',
 'YUCAIPA',
 'YUBA-SUTTER TRANSIT AUTHORITY',
 'YUBA COUNTY',
 'YUBA CITY',
 'YREKA CITY',
 'YOUNTVILLE',
 'YOSEMITE NATIONAL PARK',
 'YOSEMITE AREA REGIONAL TRANSPORTATION SYSTEM JPA',
 'YORBA LINDA',
 'YOLO COUNTY TRANSPORTATION DISTRICT',
 'YOLO COUNTY',
 'WOODSIDE',
 'WOODLAND',
 'WOODLAKE',
 'WINTERS',
 'WINDSOR',
 'WILLOWS',
 'WILLOW CREEK COMMUNITY SERVICES DISTRICT',
 'WILLITS',
 'WILLIAMS',
 'WILDOMAR',
 'WHITTIER',
 'WHEATLAND',
 'WESTMORLAND',
 'WESTMINSTER',
 'WESTLAKE VILLAGE',
 'WESTERN SHASTA RESOURCE CONSERVATION DISTRICT',
 'WESTERN RIVERSIDE COUNCIL OF GOVERNMENTS',
 'WESTERN PACIFIC RAILROAD',
 'WESTERN CONTRA COSTA TRANSIT AUTHORITY',
 'WEST SACRAMENTO',
 'WEST HOLLYWOOD',
 'WEST COVINA',
 'WEED',
 'WATSONVILLE',
 'WATERSHED CONSERVATION AUTHORITY',
 'WATERFORD',
 'WASCO',
 'WALNUT CREEK',
 'WALNUT',
 'VOLCAN MOUNTAIN PRESERVE FOUNDATION',
 'VISTA',
 'VISALIA UNIFIED SCHOOL DISTRICT',
 'VISALIA ELECTRIC RAILROAD COMPANY',
 'VISALIA',
 'VILLA PARK',


In [39]:
pattern = '|'.join(locode_names)

In [40]:
test_df.project_title.str.contains(pattern)

/tmp/ipykernel_2471/2618956443.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


203     False
324      True
448      True
477      True
560     False
589     False
600      True
633      True
667      True
742     False
983     False
1029    False
Name: project_title, dtype: bool

In [41]:
### https://stackoverflow.com/questions/68869434/create-an-pandas-column-if-a-string-from-a-list-matches-from-another-column
#test_df["agencies_found"] = test_df["project_title"].apply(lambda x: next((a for a in locode_names if a in x), ""))
test_df["agencies_found2"] = test_df["project_title"].map(lambda s: next((name for name in locode_names if name in s), np.nan))

/tmp/ipykernel_2471/35383681.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [42]:
test_df["agencies_found2"] = test_df["agencies_found2"].str.title()

/tmp/ipykernel_2471/2687676984.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [43]:
test_df

,fmis_transaction_date,project_number,implementing_agency,summary_recipient_defined_text_field_1_value,program_code,program_code_description,recipient_project_number,improvement_type,improvement_type_description,project_title,obligations_amount,congressional_district,district,county_code,county_name,county_name_title,implementing_agency_locode,rtpa_name,mpo_name,project_title_new,agencies_found2
203,44686,6204134,Caltrans,S6204MTC,Y230,Surface Transportation Block Grant (STBG) Program,0419000044L,15,Preliminary Engineering,"ALONG THE I-880 CORRIDOR SF BAY AREA: REGIONWIDE: IMPLEMENT A COLLECTIVE APPROACH TO FREEWAY OPERATIONS AND MANAGEMENT, INCLUDING COMMUNICATIONS NETW",1740000.0,Cong Dist 13,4.0,1,Multi-County,Multi-County,6204.0,CT-ADMIN,CT-ADMIN,Caltrans Preliminary Engineering Projects,NaN
324,44728,6205024,Caltrans,S6205SBCAG,YS30,Highway Safety Improvement Program (HSIP),0517000039L,21,Safety,ON SR 135 (BROADWAY) IN SANTA MARIA FROM FOSTER RD. TO PREISKER LN. INCREASE VISIBILITY OF TRAFFIC SIGNAL INDICATIONS BY INSTALLING ADDITIONAL TRAFFI,795900.0,Cong Dist 24,5.0,83,Multi-County,Multi-County,6205.0,CT-ADMIN,CT-ADMIN,Install Traffic Signals in Santa Maria,Santa Maria
448,44782,6203067,Caltrans,L6203SACOG,Y230,Surface Transportation Block Grant (STBG) Program,0322000243L,4,4R - No Added Capacity,"IN ROCKLIN: BETWEEN SR-65 (PM 4.1) AND ROCKLIN RD. (PM 6.0); CONSTRUCT I-80 EASTBOUND AUXILIARY LANE, INCLUDING TWO-LANE OFF-RAMP. IN ROSEVILLE: BET",3000000.0,Cong Dist 4,3.0,61,Multi-County,Multi-County,6203.0,CT-ADMIN,CT-ADMIN,Construct Auxiliary Lane in Roseville,Roseville
477,44796,6211149,Caltrans,S6211SANDAG,Y003,Projects to Reduce PM 2.5 Emissions,1119000022L,44,Other,IN SAN DIEGO COUNTY IN SAN DIEGO FROM 0.6 MILE NORTH FROM DEL MAR HEIGHTS ROAD OVERCROSSING TO VILLA DE LA VALLE OVERCROSSING. RESTORE WETLANDS AT SA,33824168.0,Cong Dist 52,11.0,73,Multi-County,Multi-County,6211.0,CT-ADMIN,CT-ADMIN,Restore Wetlands in San Diego County,San Diego County
560,44815,6203056,Caltrans,S6203SACOG,Y400,Congestion Mitigation & Air Quality Improvement (CMAQ),0316000113L,15,Preliminary Engineering,"SR-51 (CAPITAL CITY) CORRIDOR IMPROVEMENTS FROM J ST TO ARDEN WAY. WIDEN AND CONSTRUCT BUS/CARPOOL LANES, TRANSITION/AUXILIARY & NB CLASS I BIKE PATH",2000000.0,Cong Dist 6,3.0,67,Multi-County,Multi-County,6203.0,CT-ADMIN,CT-ADMIN,Caltrans Construct Bike Lanes,NaN
589,44817,6207070,Caltrans,S6207SCAG,Y400,Congestion Mitigation & Air Quality Improvement (CMAQ),0713000007L,1 | 17,New Construction Roadway | Construction Engineering,RT. 10 : FROM CITRUS AVE. TO RT. 57 CONSTRUCT HOV LANE IN BOTH DIRECTIONS. (TC),8165020.0,Cong Dist 32 | Cong Dist 35 | Cong Dist 39,7.0,37,Multi-County,Multi-County,6207.0,CT-ADMIN,CT-ADMIN,Caltrans Construct HOV Lane,NaN
600,44818,6208035,Caltrans,S6208SCAG,Y230,Surface Transportation Block Grant (STBG) Program,0820000147L,15,Preliminary Engineering,"I-10 CORRIDOR CONTRACT 2: THE PROJECT WILL PROVIDE ONE EXPRESS LANES IN EACH DIRECTION FROM JUST EAST OF I-15 TOPEPPER AVENUE IN COLTON, CONNECTING T",6000000.0,Cong Dist 35,8.0,71,Multi-County,Multi-County,6208.0,CT-ADMIN,CT-ADMIN,Express Lanes in Colton,Colton
633,44825,6203069,Caltrans,L6203BCAG,Y400,Congestion Mitigation & Air Quality Improvement (CMAQ),0316000060L,15 | 16,Preliminary Engineering | Right of Way,"IN AND NEAR GRIDLEY, FROM HOLLIS LANE TO NORTH OF FORD AVENUE. DIST-CO-RTE: 03-BUT-99 PM/PM: R3.1/5.0 REHABILITATE PAVEMENT, UPGRADE TRANSPORTATION M",700000.0,Cong Dist 1,3.0,7,Multi-County,Multi-County,6203.0,CT-ADMIN,CT-ADMIN,Upgrade Pavement Rehabilitation in Gridley,Gridley
667,44827,2023000,Unknown,nan,YR10,State RTP Administration,nan,44,Other,2023 Rec Trails Project: State Parks Incurred Administrative Expenses,241760.0,Cong Dist 0,nan,999,Unknown,Unknown,nan,None,None,Recreational Trails Project in State Parks,State Parks
742,44879,6212023,Caltrans,S6212SCAG,Y230,Surface Transportation Block Grant (STBG) Program,1213000084L,15,Preliminary Engine

In [44]:
# agg>>filter(_["Project Title New"].str.contains('Unknown'))

In [45]:
agg>>filter(_["implementing_agency"].str.contains('Unknown'))

,fmis_transaction_date,project_number,implementing_agency,summary_recipient_defined_text_field_1_value,program_code,program_code_description,recipient_project_number,improvement_type,improvement_type_description,project_title,obligations_amount,congressional_district,district,county_code,county_name,county_name_title,implementing_agency_locode,rtpa_name,mpo_name,project_title_new
667,44827,2023000,Unknown,nan,YR10,State RTP Administration,nan,44,Other,2023 Rec Trails Project: State Parks Incurred Administrative Expenses,241760.0,Cong Dist 0,nan,999,Unknown,Unknown,nan,None,None,Recreational Trails Project in State Parks
